In [ ]:
! cd /root/
! mkdir .kaggle
# ! cd /root/.kaggle/
! touch .kaggle/kaggle.json
# ! rm kaggle.json
! ls -la

In [ ]:
api_token = {"username":"kedermallick","key":"cf972c2245917c60af89c3dc85131e18"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!kaggle datasets download -d msambare/fer2013

In [ ]:
! unzip /root/fer2013.zip -d /kaggle/working/

In [ ]:
! sudo pip install keras

In [30]:
import tensorflow.keras as keras
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalAveragePooling2D, BatchNormalization,AveragePooling2D
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import os,cv2
train_dataset_path="/kaggle/input/fer2013/train/"
train_data_path=os.listdir(os.path.join(train_dataset_path))
# print(train_data_path)

angry_img_path=os.path.join(train_dataset_path,train_data_path[0])
# print(angry_img_path.)
angry_img=os.path.join(angry_img_path,os.listdir(os.path.join(angry_img_path))[123])
image=cv2.imread(angry_img)
# plt.imshow(image)
print(image.shape)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=32
seed=40
train_datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    rescale=1./255, 
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.20
)
validation_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.20
)
train_generator=train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(224, 224),
    color_mode='rgb', 
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    interpolation='nearest',
    subset='training'
)
validation_generator=validation_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(224, 224),
    color_mode='rgb', 
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    interpolation='nearest',
    subset='validation'
)
print(train_generator)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
    images,label=train_generator.next()
    ax=plt.subplot(3,3,i+1)
    plt.imshow(images[0])
    plt.title(label=label)

In [36]:
from keras.layers import Input
from keras.layers.merge import concatenate

input_img_shape=Input(shape=(299,299,3))
classes=7
channel_no=3
model_weight_path="inception_v3_weights_tf_dim_ordering_tf_kernels.h5"

In [3]:
## Inceprion v3 implementation 
def inception_init_modelue(x,f1,f2,f3):
#     x input layer
#     1X1 convolution layer
    conv1=Conv2D(f1,(1,1),padding='same',activation='relu')(x)
#     3X3 convolution layer
    conv2=Conv2D(f2,(3,3),padding='same',activation='relu')(x)
#     5X5 convolution layer
    conv3=Conv2D(f3,(5,5),padding='same',activation='relu')(x)
#     3X3 maxPooling layer
    pool1=MaxPooling2D((3,3),stride=(1,1),padding='same')(x)
#     concatenate all layer to prodece finnal filtered output
    output=concatenate([conv1,conv2,conv3,pool1])
    return output

In [4]:

#  In case of inception v3 each and every convolution layer must have BatchNormalization layer
#     relu activation function

def custom_conv2D(x,filter_size,kernel_row,kernel_col,stride=(1,1),padding='same'):
    x=Conv2D(filter_size,(kernel_row,kernel_col),stride,padding=padding)(x)
    x=BatchNormalization(axis=3,scale=False)(x)
    x=Activation('relu')(x)
    return x

In [21]:
def inception_block_a(x):
#     single direct 1X1 covolution
    level_0_conv1=custom_conv2D(x,64,1,1)
    
    branch5X5=custom_conv2D(x,48,1,1)
    branch5X5=custom_conv2D(branch5X5,64,5,5)
    
    branch3X3dbl=custom_conv2D(x,64,1,1)
    branch3X3dbl=custom_conv2D(branch3X3dbl,96,3,3)
    branch3X3dbl=custom_conv2D(branch3X3dbl,96,3,3)
    
    branch_pool=AveragePooling2D((3,3),strides=(1,1),padding='same')(x)
    branch_pool=custom_conv2D(branch_pool,32,1,1)
    
    x=concatenate([level_0_conv1,branch5X5,branch3X3dbl,branch_pool],axis=channel_no)
    
    return x

In [34]:
# reduction block a
def reduction_block_a(x):
    branch3X3=custom_conv2D(x,384,3,3,stride=(2,2),padding='valid')

    branch3X3dbl=custom_conv2D(x,64,1,1)
    branch3X3dbl=custom_conv2D(branch3X3dbl,96,3,3)
    branch3X3dbl=custom_conv2D(branch3X3dbl,96,3,3,stride=(2,2),padding='valid')

    # branch_pool=MaxPooling2D((3,3),strides=(2,2))(x)
    # if we do so shape will nbe missmatched so do a custom_conv2D
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = custom_conv2D(branch_pool, 64, 1, 1)

    x=concatenate([branch3X3,branch3X3dbl],axis=channel_no)

    return x

In [10]:
def inception_block_b(x):
    branch1=custom_conv2D(x,192,1,1)

    # define 7X7 convolution as 1X7 and 7X1 combinations

    branch7X7=custom_conv2D(x,128,1,1)
    branch7X7=custom_conv2D(branch7X7,128,1,7)
    branch7X7=custom_conv2D(branch7X7,192,7,1)

    # tow parallel layer of prevoious layer

    branch7X7dbl=custom_conv2D(x,128,1,1)
    branch7X7dbl=custom_conv2D(branch7X7dbl,128,1,7)
    branch7X7dbl=custom_conv2D(branch7X7dbl,128,7,1)
    branch7X7dbl=custom_conv2D(branch7X7dbl,128,1,7)
    branch7X7dbl=custom_conv2D(branch7X7dbl,192,7,1)
    #out put layer will conatin 192 filters

    branch_pool=AveragePooling2D((3,3),strides=(1,1),padding='same')(x)
    branch_pool=custom_conv2D(branch_pool,192,1,1)

    x=concatenate([branch1,branch7X7,branch7X7dbl,branch_pool],axis=channel_no)

    return x


In [33]:
def reduction_block_b(x):
    branch3X3=custom_conv2D(x,192,1,1)
    branch3X3=custom_conv2D(branch3X3,320,3,3,stride=(3,3),padding='valid')

    branch7X7X3=custom_conv2D(x,192,1,1)
    branch7X7X3=custom_conv2D(branch7X7X3,192,1,7)
    branch7X7X3=custom_conv2D(branch7X7X3,192,7,1)
    branch7X7X3=custom_conv2D(branch7X7X3,320,3,3,stride=(3,3),padding='valid')

    branch_pool=MaxPooling2D((3,3),strides=(2,2))(x)
    branch_pool = custom_conv2D(branch_pool, 192, 1, 1)
    x= concatenate([branch3X3,branch7X7X3],axis=channel_no)

    return x

In [13]:
def inception_block_c(x):
    branch1=custom_conv2D(x,320,1,1)

    branch3X3=custom_conv2D(x,384,1,1)
    branch3X3X1=custom_conv2D(branch3X3,384,1,3)
    branch3X3X2=custom_conv2D(branch3X3,384,1,3)
    branch3X3=concatenate([branch3X3X1,branch3X3X2],axis=channel_no)

    branch3X3dbl=custom_conv2D(x,448,1,1)
    branch3X3dbl=custom_conv2D(branch3X3dbl,384,3,3)
    branch3X3dbl_1=custom_conv2D(branch3X3dbl,384,1,3)
    branch3X3dbl_2=custom_conv2D(branch3X3dbl,384,3,1)
    branch3X3dbl=concatenate([branch3X3dbl_1,branch3X3dbl_2],axis=channel_no)

    branch_pool=AveragePooling2D((3,3),strides=(1,1),padding='same')(x)
    branch_pool=custom_conv2D(branch_pool,192,1,1)

    x= concatenate([branch1,branch3X3,branch3X3dbl,branch_pool],axis=channel_no)

    return x


In [39]:
x=custom_conv2D(input_img_shape,32,3,3,stride=(2,2),padding='valid')
x=custom_conv2D(x,32,3,3,padding='valid')
x=custom_conv2D(x,64,3,3)
x=MaxPooling2D((3,3),strides=(2,2))(x)


x=custom_conv2D(x,80,1,1,padding='valid')
x=custom_conv2D(x,192,3,3,padding='valid')
x=MaxPooling2D((3,3),strides=(2,2))(x)

# inception init block a
# size of 3
x=inception_block_a(x)
x=inception_block_a(x)
x=inception_block_a(x)

# reduction block a

x=reduction_block_a(x)

# inception init block b
x=inception_block_b(x)
x=inception_block_b(x)
x=inception_block_b(x)
x=inception_block_b(x)

# reduction block b
x=reduction_block_b(x)

# inception block c

x=inception_block_c(x)
x=inception_block_c(x)

x=GlobalAveragePooling2D(name='avg_pool')(x)

x=Dense(classes,activation='softmax',name='predictions')(x)

# create model
models=Model(input_img_shape,x,name="my_inception_v3")
#  
#return and save weights
# models.load_weights(model_weight_path,by_name=True,skip_mismatch=True)

models.summary()

Model: "my_inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_829 (Conv2D)            (None, 149, 149, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_828 (Batch  (None, 149, 149, 32  96         ['conv2d_829[0][0]']             
 Normalization)                 )                                                   

**why load models weights?**

Model weights are all the parameters (including trainable and non-trainable) of the model which are in 
turn all the parameters used in the layers of the model. 
Now,By the whole code written here we actually declare the models architechture or the layer of the
inception model.But we have to load the trainable and non trainable parameter through the weights(again
a model).
So we use the pretrained iamgent model here.

### actual inception model link
```
https://github.com/keras-team/keras/blob/master/keras/applications/inception_v3.py
```
### References :
1. https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202

In [ ]:
epochs = 50

# callbacks = [
#     keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
# ]
models.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
history= models.fit_generator(
    steps_per_epoch=22968//batch_size,
    generator=train_generator,
    validation_data=validation_generator,
    validation_steps=5741//batch_size,
    epochs=epochs
)